In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
dataset = pd.read_csv('datas/diabetes.csv')

In [4]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
dataset.isnull()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
763,False,False,False,False,False,False,False,False,False
764,False,False,False,False,False,False,False,False,False
765,False,False,False,False,False,False,False,False,False
766,False,False,False,False,False,False,False,False,False


In [6]:
dataset.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [7]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 8].values

In [8]:
print(X.shape)
print(y.shape)

(768, 8)
(768,)


In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.01, random_state = 0)

In [10]:
sc = StandardScaler()
Xtrain = sc.fit_transform(Xtrain)
Xtest = sc.fit_transform(Xtest)
X = sc.fit_transform(X)

In [11]:
print(Xtrain.shape)
print(Xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(760, 8)
(8, 8)
(760,)
(8,)


In [12]:
Xtrain = torch.from_numpy(Xtrain.astype('float32'))
Xtest = torch.from_numpy(Xtest.astype('float32'))

In [13]:
ytrain = torch.squeeze(torch.from_numpy(ytrain.astype('float32')))
ytest = torch.squeeze(torch.from_numpy(ytest.astype('float32')))

In [14]:
print(Xtrain.size())
print(Xtest.size())
print(ytrain.size())
print(ytest.size())

torch.Size([760, 8])
torch.Size([8, 8])
torch.Size([760])
torch.Size([8])


In [15]:
Xtrain = Xtrain.to(device)
Xtest = Xtest.to(device)
ytrain = ytrain.to(device)
ytest = ytest.to(device)

In [39]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(8, 15)
        self.layer2 = nn.Linear(15, 10)
        self.layer3 = nn.Linear(10,1)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return torch.sigmoid(x)
    
net = Net().to(device)
print(net)

Net(
  (layer1): Linear(in_features=8, out_features=15, bias=True)
  (layer2): Linear(in_features=15, out_features=10, bias=True)
  (layer3): Linear(in_features=10, out_features=1, bias=True)
)


In [40]:
loss = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr= 0.1)

In [50]:
EPOCHS = 1000

for i in range(EPOCHS):
    ypred = net(Xtrain)
    ypred = torch.squeeze(ypred)
    training_loss = loss(ypred, ytrain)
    
    optimizer.zero_grad()
    
    training_loss.backward()
    
    optimizer.step()
    
    if i %100 ==0:
        print('The running loss is', training_loss)

The running loss is tensor(0.5526, grad_fn=<BinaryCrossEntropyBackward>)
The running loss is tensor(0.7477, grad_fn=<BinaryCrossEntropyBackward>)
The running loss is tensor(0.7128, grad_fn=<BinaryCrossEntropyBackward>)
The running loss is tensor(0.7104, grad_fn=<BinaryCrossEntropyBackward>)
The running loss is tensor(0.7082, grad_fn=<BinaryCrossEntropyBackward>)
The running loss is tensor(0.5903, grad_fn=<BinaryCrossEntropyBackward>)
The running loss is tensor(0.5565, grad_fn=<BinaryCrossEntropyBackward>)
The running loss is tensor(0.5562, grad_fn=<BinaryCrossEntropyBackward>)
The running loss is tensor(0.5550, grad_fn=<BinaryCrossEntropyBackward>)
The running loss is tensor(0.6469, grad_fn=<BinaryCrossEntropyBackward>)


In [51]:
with torch.no_grad():
    ypred = net(Xtrain)

In [52]:
y_train = ytrain.detach().numpy().reshape(-1, 1)
y_pred = ypred.detach().numpy()

In [53]:
for i in range(y_pred.shape[0]):
    if y_pred[i]>=0.5:
        y_pred[i] =1
    else:
        y_pred[i] = 0

In [54]:
print(accuracy_score(y_train, y_pred))

0.9263157894736842


In [55]:
with torch.no_grad():
    ypred = net(Xtest)

In [56]:
Ytrain = ytest.detach().numpy()
Ypred = ypred.detach().numpy()

In [57]:
for i in range(Ypred.shape[0]):
    if Ypred[i]>=0.5:
        Ypred[i] =1
    else:
        Ypred[i] = 0

In [58]:
print(accuracy_score(Ytrain, Ypred))

0.875


In [59]:
name = 'Model1.pth'
torch.save(net.state_dict(), name)

In [ ]:
net = Net()
net.load_state_dict(torch.load(name))